In [47]:
import os
import pandas as pd
import numpy as np
import gc

In [48]:
os.chdir("C:/Users/shaur/Downloads/commonlit")
os.getcwd()

'C:\\Users\\shaur\\Downloads\\commonlit'

In [49]:
prompts_train = pd.read_csv("prompts_train.csv")
prompts_test = pd.read_csv("prompts_test.csv")

summaries_train = pd.read_csv("summaries_train.csv")
summaries_test = pd.read_csv("summaries_test.csv")

sample_submission = pd.read_csv("sample_submission.csv")

prompts_train

,prompt_id,prompt_question,prompt_title,prompt_text
0,39c16e,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \r\nAs the sequel to what has alrea...
1,3b9047,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...
2,814d6b,Summarize how the Third Wave developed over su...,The Third Wave,Background \r\nThe Third Wave experiment took ...
3,ebad26,Summarize the various ways the factory would u...,Excerpt from The Jungle,"With one member trimming beef in a cannery, an..."


In [50]:
train = summaries_train.merge(prompts_train, how="left", on="prompt_id")
test = summaries_test.merge(prompts_test, how="left", on="prompt_id")

In [51]:
import warnings
import logging
import shutil
import json
import transformers
from transformers import AutoModel, AutoTokenizer, AutoConfig, AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding, TrainingArguments, Trainer
from datasets import Dataset, load_dataset, load_from_disk, load_metric, disable_progress_bar
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GroupShuffleSplit, GroupKFold
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [52]:
warnings.simplefilter("ignore")
logging.disable(logging.ERROR)
disable_progress_bar()

In [53]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    rmse = mean_squared_error(labels, predictions, squared=False)
    return {"rmse": rmse}

def compute_mcrmse(eval_pred):
    """
    Calculates mean columnwise root mean squared error
    https://www.kaggle.com/competitions/commonlit-evaluate-student-summaries/overview/evaluation
    """
    preds, labels = eval_pred

    col_rmse = np.sqrt(np.mean((preds - labels) ** 2, axis=0))
    mcrmse = np.mean(col_rmse)

    return {
        "content_rmse": col_rmse[0],
        "wording_rmse": col_rmse[1],
        "mcrmse": mcrmse,
    }

In [54]:
def compt_score(content_true, content_pred, wording_true, wording_pred):
    content_score = mean_squared_error(content_true, content_pred)**(1/2)
    wording_score = mean_squared_error(wording_true, wording_pred)**(1/2)

    return (content_score + wording_score)/2

def seed_everything(seed: int):
    import random, os
    import numpy as np
    import torch

    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True


seed_everything(seed=42)

In [55]:
splitter = GroupShuffleSplit(test_size=.2, n_splits=4, random_state=42)
split = splitter.split(train, groups=train['prompt_id'])
train_ind, val_ind = next(split)

train_split = train.iloc[train_ind]
val_split = train.iloc[val_ind]

In [56]:
train_split.prompt_id.value_counts()

prompt_id
39c16e    2057
ebad26    1996
814d6b    1103
Name: count, dtype: int64

In [57]:
val_split.prompt_id.value_counts()

prompt_id
3b9047    2009
Name: count, dtype: int64

### Function for train and infer

In [58]:
def train_n_infer(train,
                  val,
                  test_content,
                  model_name,
                  num_layers_to_freeze,
                  learning_rate,
                  per_device_train_batch_size,
                  per_device_eval_batch_size,
                  num_train_epochs,
                  ss,
                  hidden_dropout_prob,
                  attention_probs_dropout_prob,
                  fold):
    
    train_content = train[["text", "content", "wording"]]
    val_content = val[["text", "content", "wording"]]
    test_ = test_content[["text"]]

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model_config = AutoConfig.from_pretrained(model_name)
    model_config.update({
        "hidden_dropout_prob": hidden_dropout_prob,
        "attention_probs_dropout_prob": attention_probs_dropout_prob,
        "num_labels": 2,
        "problem_type": "regression",
    })
    seed_everything(seed=42)
    model = AutoModelForSequenceClassification.from_pretrained(model_name, config=model_config).to(device)
    

    layer_count=0
    for name, param in model.named_parameters():
        if layer_count<num_layers_to_freeze:
            param.requires_grad=False
            layer_count+=1
        else: break
    
    train_dataset_content = Dataset.from_pandas(train_content, preserve_index=False)
    val_dataset_content = Dataset.from_pandas(val_content, preserve_index=False)
    test_dataset = Dataset.from_pandas(test_, preserve_index=False)

    def tokenize_function(examples):
        labels = [examples["content"], examples["wording"]]
        tokenized = tokenizer(examples["text"],
                            padding=False,
                            truncation=True,
                            )
        return {
            **tokenized,
            "labels": labels,
            }

    def tokenize_function_test(examples):
        tokenized = tokenizer(examples["text"],
                            padding=False,
                            truncation=True,
                            )
        return tokenized

    train_tokenized_datasets_content = train_dataset_content.map(tokenize_function, batched=False)
    val_tokenized_datasets_content = val_dataset_content.map(tokenize_function, batched=False)
    test_tokenized_dataset = test_dataset.map(tokenize_function_test, batched=False)

    model_dir = f"./Results/{model_name}_results/fold {fold}"
    os.makedirs(model_dir, exist_ok=True)

    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    training_args = TrainingArguments(
        output_dir = model_dir,
        load_best_model_at_end = True,
        learning_rate = learning_rate,
        per_device_train_batch_size=per_device_train_batch_size,
        per_device_eval_batch_size=per_device_eval_batch_size,
        num_train_epochs=num_train_epochs,
        save_strategy="steps",
        evaluation_strategy="steps",
        greater_is_better=False,
        metric_for_best_model="mcrmse",
        eval_steps=ss,
        save_steps=ss,
        report_to="none",
        save_total_limit=3
    )

    trainer_content = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_tokenized_datasets_content,
            eval_dataset=val_tokenized_datasets_content,
            tokenizer=tokenizer,
            compute_metrics=compute_mcrmse,
            data_collator=data_collator
        )
    
    trainer_content.train()

    best_check = os.listdir(model_dir)[0]
    model_content = AutoModelForSequenceClassification.from_pretrained(f"{model_dir}/{best_check}")
    model_content.eval()

    test_args = TrainingArguments(
        output_dir=model_dir,
        do_train = False,
        do_predict = True,
        per_device_eval_batch_size = per_device_eval_batch_size,
        dataloader_drop_last = False,
    )

    # init trainer
    infer_content = Trainer(
                  model = model_content,
                  tokenizer=tokenizer,
                  data_collator=data_collator,
                  args = test_args)

    val_results_content = infer_content.predict(val_tokenized_datasets_content)[0]
    test_results_content = infer_content.predict(test_tokenized_dataset)[0]


    model_content.save_pretrained(model_dir)
    tokenizer.save_pretrained(model_dir)
    
    model.cpu()
    model_content.cpu()
    del model
    del model_content
    gc.collect()
    torch.cuda.empty_cache()


    return val_results_content, test_results_content

### GroupKFold

In [59]:
def get_oof_pred_n_test(train,
                        test,
                        model_name,
                        n_splits,
                        num_layers_to_freeze,
                        learning_rate,
                        per_device_train_batch_size,
                        per_device_eval_batch_size,
                        num_train_epochs,
                        ss,
                        hidden_dropout_prob,
                        attention_probs_dropout_prob):
    
    kf = GroupKFold(n_splits=n_splits)
    oof_content = np.zeros((len(train), 2))
    test_pred_content = np.zeros((len(test), 2))

    for i, (train_indx, val_indx) in enumerate(kf.split(train, groups=train["prompt_id"])):
        print(f"fold {i}:")
        train_ = train.iloc[train_indx]
        val_ = train.iloc[val_indx]

        val_res_content, test_res_content = train_n_infer(train_,
                                                          val_,
                                                          test,
                                                          model_name,
                                                          num_layers_to_freeze,
                                                          learning_rate,
                                                          per_device_train_batch_size,
                                                          per_device_eval_batch_size,
                                                          num_train_epochs,
                                                          ss,
                                                          hidden_dropout_prob,
                                                          attention_probs_dropout_prob,
                                                          i)
        oof_content[val_indx] = val_res_content
        test_pred_content += test_res_content/n_splits
    
    oof_train = pd.DataFrame(oof_content, columns=[f"content_pred_{model_name}", f"wording_pred_{model_name}"])
    test_pred = pd.DataFrame(test_pred_content, columns=[f"content_pred_{model_name}", f"wording_pred_{model_name}"])

    cv_metric = compute_mcrmse((oof_train.values, train[["content", "wording"]]))
    print(f"cv mcrmse: {cv_metric}")
    
    with open(f"./Results/{model_name}_results/cv_metric.json", "w") as outfile:
        json.dump(cv_metric, outfile)
        
    oof_train.to_csv(f"./Results/{model_name}_results/oof_train.csv", index=False)

    return oof_train, test_pred


In [60]:
class CFG:
    model_name="deberta_v3_base"
    learning_rate=1.5e-5
    num_train_epochs=3
    num_layers_to_freeze=20
    n_splits=4
    batch_size=12
    random_seed=42
    save_steps=100
    max_length=512
    per_device_train_batch_size=2
    per_device_eval_batch_size=2
    hidden_dropout_prob=0.005
    attention_probs_dropout_prob=0.005

In [61]:
oof_train, pred_test = get_oof_pred_n_test(train,
                                           test,
                                           CFG.model_name,
                                           CFG.n_splits,
                                           CFG.num_layers_to_freeze,
                                           CFG.learning_rate,
                                           CFG.per_device_train_batch_size,
                                           CFG.per_device_eval_batch_size,
                                           CFG.num_train_epochs,
                                           CFG.save_steps,
                                           CFG.hidden_dropout_prob,
                                           CFG.attention_probs_dropout_prob
                                           )

fold 0:


  0%|          | 7/7662 [14:07:17<15442:59:45, 7262.55s/it]
                                                  
  1%|▏         | 100/7662 [00:30<10:56, 11.52it/s] 

{'eval_loss': 0.4981825351715088, 'eval_content_rmse': 0.5610414743423462, 'eval_wording_rmse': 0.8255892395973206, 'eval_mcrmse': 0.6933153867721558, 'eval_runtime': 20.6148, 'eval_samples_per_second': 99.783, 'eval_steps_per_second': 49.916, 'epoch': 0.04}


                                                    
  3%|▎         | 200/7662 [01:03<11:08, 11.16it/s] 

{'eval_loss': 0.5217586159706116, 'eval_content_rmse': 0.719721794128418, 'eval_wording_rmse': 0.7249259352684021, 'eval_mcrmse': 0.7223238945007324, 'eval_runtime': 20.8727, 'eval_samples_per_second': 98.55, 'eval_steps_per_second': 49.299, 'epoch': 0.08}


...

{'eval_loss': 0.5359641313552856, 'eval_content_rmse': 0.7341975569725037, 'eval_wording_rmse': 0.7299876809120178, 'eval_mcrmse': 0.7320926189422607, 'eval_runtime': 12.6302, 'eval_samples_per_second': 87.33, 'eval_steps_per_second': 43.705, 'epoch': 2.97}


100%|██████████| 9093/9093 [34:50<00:00,  4.35it/s]


{'train_runtime': 2090.8739, 'train_samples_per_second': 8.698, 'train_steps_per_second': 4.349, 'train_loss': 0.19879677874360505, 'epoch': 3.0}


100%|██████████| 2/2 [00:00<00:00, 140.95it/s]


cv mcrmse: {'content_rmse': 0.47593582945081453, 'wording_rmse': 0.6257460187358439, 'mcrmse': 0.5508409240933292}


In [64]:
sample_submission["content"] = pred_test.values[:, 0]
sample_submission["wording"] = pred_test.values[:, 1]

In [65]:
sample_submission.to_csv("submission.csv", index=False)

In [66]:
sample_submission

,student_id,content,wording
0,000000ffffff,-1.469326,-1.329178
1,111111eeeeee,-1.470571,-1.328479
2,222222cccccc,-1.467416,-1.322249
3,333333dddddd,-1.474524,-1.330803
